In [1]:
import Utils
from Utils import read_txt
import tools
import copy
import os
from PIL import Image
import torch
import warnings
import numpy as np
from tqdm.notebook import tqdm
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
---------------------
import easydict
import time

# Fix Seed
torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")

# Fix CPU Limitation
torch.set_num_threads(3)

### DataLoader

In [2]:
class multi_modal_dataset(Dataset):
    def __init__(self, classification_dir, segmentation_dir, txt_COVID, txt_NonCOVID,
                 audio_dir, select_num,
                 lateral_map=1, min_seg=0.01, transform=None):
        
        # Dataset Root Directory
        self.classification_dir = classification_dir
        self.segmentation_dir = segmentation_dir
        self.audio_dir = audio_dir
        
        # Subject
        self.txt_path = [txt_COVID, txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.audio_classes = ['pos', 'neg']
        self.num_cls = len(self.classes)

        self.img_list = []
        self.segment_list = []
        self.audio_list = []
        
        self.min_seg = min_seg
        self.select_num = select_num
        
        for c in range(self.num_cls):
            # Classification List
            cls_list = [[os.path.join(self.classification_dir, self.classes[c], item), c] for item in
                        read_txt(self.txt_path[c])]
            self.img_list += cls_list
            
            # Audio List
            a_list = [[os.path.join(audio_dir, self.audio_classes[c], item), c] for item in os.listdir(os.path.join(audio_dir, self.audio_classes[c]))]
            self.audio_list += a_list
            
        # Split Image COVID & Non-COVID
        self.img_list = np.array(self.img_list)
        pos_index = (self.img_list[:,1]=='0')
        neg_index = (self.img_list[:,1]=='1')
        pos_img_list = self.img_list[:, 0][pos_index]
        neg_img_list = self.img_list[:, 0][neg_index]

        # Select Image List
        select_pos_img_list = np.random.choice(pos_img_list, self.select_num)
        select_neg_img_list = np.random.choice(neg_img_list, self.select_num)

        # Mapping Segment List
        select_pos_seg_list = []
        select_neg_seg_list = []
        for s_p_i_l in select_pos_img_list:
            select_pos_seg_list.append(os.path.join(segmentation_dir, s_p_i_l.split('/')[-2], "lateral_map" + str(lateral_map),
                              s_p_i_l.split('/')[-1].replace('.jpg', '.png')))
        for s_n_i_l in select_neg_img_list:
            select_neg_seg_list.append(os.path.join(segmentation_dir, s_n_i_l.split('/')[-2], "lateral_map" + str(lateral_map),
                              s_n_i_l.split('/')[-1].replace('.jpg', '.png')))

        # Split Audio COIVD & Non-COVID
        self.audio_list = np.array(self.audio_list)
        pos_index = (self.audio_list[:,1]=='0')
        neg_index = (self.audio_list[:,1]=='1')
        pos_audio_list = self.audio_list[:, 0][pos_index]
        neg_audio_list = self.audio_list[:, 0][neg_index]

        # Select Audio List
        select_pos_audio_list = np.random.choice(pos_audio_list, select_num)
        select_neg_audio_list = np.random.choice(neg_audio_list, select_num)
        
        # Make DICT
        self.data_list = []
        for i in range(self.num_cls):
            for j in range(self.select_num):
                if i == 0:
                    data_dict = {'img': select_pos_img_list[j],
                                 'seg': select_pos_seg_list[j],
                                 'audio': select_pos_audio_list[j],
                                 'label': i}
                else:
                    data_dict = {'img': select_neg_img_list[j],
                                 'seg': select_neg_seg_list[j],
                                 'audio': select_neg_audio_list[j],
                                 'label': i}
                self.data_list.append(data_dict)

        self.transform = transform

    def __len__(self):
        return self.select_num*2

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Original Data
        img_path = self.data_list[idx]['img']
        image = Image.open(img_path).convert('RGB')
        image = image.resize((256, 256))

        # Segmentation Data
        seg_path = self.data_list[idx]['seg']
        seg = Image.open(seg_path).convert('RGB')
        seg = seg.rotate(-90, expand=True)
        seg = seg.resize((256, 256))
        
        # Audio Data
        audio_path = self.data_list[idx]['audio']
        audio = Image.open(audio_path).convert('RGB')

        # Mask with Original Data
        # Step 1 => Segmentation Min-Max Normalization + Min Value(Hyperparameter)
        seg_np = np.array(seg)
        seg_mask = (seg_np - seg_np.min()) / (seg_np.max() - seg_np.min()) + self.min_seg
        # Clip max = 1
        seg_mask = np.clip(seg_mask, 0, 1)

        # Step 2 => Original Data with seg_mask
        image_with_mask = np.multiply(image, seg_mask)

        # Step 3 => Change Numpy Dtype => For Using Image Preprocessing
        image_with_mask = Image.fromarray(np.uint8(image_with_mask))
        
        if self.transform:
            image_with_mask = self.transform(image_with_mask)
            audio = self.transform(audio)
        
        sample = {'img': image_with_mask,
                  'audio': audio,
                  'label': int(self.data_list[idx]['label'])}
        return sample

In [3]:
# Normalization
normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
trans = transforms.Compose([
    transforms.Resize((224)),
    transforms.ToTensor(),
    normalize
])

# Train Dataset
trainset = multi_modal_dataset(classification_dir='./dataset/image/classfication/',
                               segmentation_dir='./dataset/image/classfication/Segmentation/',
                               txt_COVID='./dataset/image/classfication/Data-split/COVID/trainCT_COVID.txt',
                               txt_NonCOVID='./dataset/image/classfication/Data-split/NonCOVID/trainCT_NonCOVID.txt',
                               audio_dir = './dataset/audio/preprocess/train/',
                               select_num=500,
                               lateral_map=3, min_seg=0.8,
                               transform=trans)

# Validation Dataset
valset = multi_modal_dataset(classification_dir='./dataset/image/classfication/',
                               segmentation_dir='./dataset/image/classfication/Segmentation/',
                               txt_COVID='./dataset/image/classfication/Data-split/COVID/valCT_COVID.txt',
                               txt_NonCOVID='./dataset/image/classfication/Data-split/NonCOVID/valCT_NonCOVID.txt',
                               audio_dir = './dataset/audio/preprocess/validation/',
                               select_num=100,
                               lateral_map=3, min_seg=0.8,
                               transform=trans)

# Test Dataset
testset = multi_modal_dataset(classification_dir='./dataset/image/classfication/',
                               segmentation_dir='./dataset/image/classfication/Segmentation/',
                               txt_COVID='./dataset/image/classfication/Data-split/COVID/testCT_COVID.txt',
                               txt_NonCOVID='./dataset/image/classfication/Data-split/NonCOVID/testCT_NonCOVID.txt',
                               audio_dir = './dataset/audio/preprocess/test/',
                               select_num=100,
                               lateral_map=3, min_seg=0.8,
                               transform=trans)

# Data Loader
multi_train_loader = DataLoader(trainset, batch_size=10, drop_last=True, shuffle=True)
multi_val_loader = DataLoader(valset, batch_size=10, drop_last=True, shuffle=False)
multi_test_loader = DataLoader(testset, batch_size=10, drop_last=True, shuffle=False)

### PreTrain Model Load

In [4]:
# DenseNet => Audio, CT BaseLine Model
print('Base Model Load....')
base_model = models.densenet169(pretrained=True)

# Image Model
print('Image Model Load....')
image_model = models.densenet169(pretrained=True)
image_model.classifier = nn.Sequential(nn.Linear(1664, 2), nn.Softmax(dim=1))
image_model.load_state_dict(torch.load('./model/single_modality/3_0.8.pt', map_location='cpu'))
image_model
image_model.eval()

# Audio Model
print('Audio Model Load....')
audio_model = models.densenet169(pretrained=True)
audio_model.classifier = nn.Linear(1664, 1)
audio_model = nn.Sequential(audio_model, nn.Sigmoid())
audio_model.load_state_dict(torch.load('./model/single_modality/audio.pt', map_location='cpu'))
audio_model
audio_model.eval()

print('All Load....')

Base Model Load....
Image Model Load....
Audio Model Load....
All Load....


### Prepare Feature Extractor

In [5]:
imgae_feature_extractor = []

for child in image_model.children():
    imgae_feature_extractor.append(nn.Sequential(child, nn.AvgPool2d((7,7))))
    imgae_feature_extractor.append(nn.Sequential(*list(child.children())[:-2], nn.AvgPool2d((7,7))))
    imgae_feature_extractor.append(nn.Sequential(*list(child.children())[:-4], nn.AvgPool2d((14,14))))
    break

In [6]:
audio_feature_extractor = []

for i, (name, module) in enumerate(audio_model[0]._modules.items()):
    audio_feature_extractor.append(nn.Sequential(module, nn.AvgPool2d((7,7))))
    audio_feature_extractor.append(nn.Sequential(*list(module.children())[:-2], nn.AvgPool2d((7,7))))
    audio_feature_extractor.append(nn.Sequential(*list(module.children())[:-4], nn.AvgPool2d((14,14))))
    break

In [7]:
# Delete PreTrain Model on Memory
del base_model
del image_model
del audio_model

### Define MFAS Search Config
- config_1 = [[1,1,0], [0,2,1], [0,0,0]]
- config_2 = [[2,0,0], [1,1,1], [0,2,0]]
- config_3 = [[0,0,1], [0,1,0]]
- config_4 = [[1,1,0], [1,1,0], [0,0,1]]
- config_5 = [[0,2,1], [2,0,0], [2,1,1]]

In [8]:
config_1 = [[0,0,1], [0,1,0]]
config_2 = [[2,0,0], [1,1,1], [0,2,0]]
config_3 = [[0,2,1], [2,0,0], [2,1,1]]

In [9]:
class mfas_fusion(nn.Module):
    def __init__(self, conf, audio_feature_extractor, image_feature_extractor, device):
        super(mfas_fusion, self).__init__()
        
        self.conf = conf
        self.device = device

        # Pre-Train Feature Extractor
        self.audio_feature_extractor = audio_feature_extractor
        self.image_feature_extractor = image_feature_extractor
        
        # Define Input Size
        input_size = [1664, 640, 256]
        
        # Defin Output Size
        self.out_size = 100
        
        self.alphas = [(input_size[conf[0]], input_size[conf[1]]) for conf in self.conf]

        # Define Fuse Layer
        self.fusion_layers = self._create_fc_layers()

        # Classification => COVID or Non-COVID
        self.central_classifier = nn.Sequential(nn.Linear(self.out_size, 2), nn.Softmax(dim=1))

    # tensor_tuple => CSF, PET, SMRI
    def forward(self, image, audio):
        # Image Feature
        image_features = [self.image_feature_extractor[0].to(self.device)(image.to(self.device)).squeeze(), 
                          self.image_feature_extractor[1].to(self.device)(image.to(self.device)).squeeze(),
                          self.image_feature_extractor[2].to(self.device)(image.to(self.device)).squeeze()]
        
        image_features = [image_features[idx] for idx in self.conf[:, 0]]

        # Audio Feature
        audio_features = [self.audio_feature_extractor[0].to(self.device)(audio.to(self.device)).squeeze(),
                          self.audio_feature_extractor[1].to(self.device)(audio.to(self.device)).squeeze(),
                          self.audio_feature_extractor[2].to(self.device)(audio.to(self.device)).squeeze()]
        
        audio_features = [audio_features[idx] for idx in self.conf[:, 1]]

        # Fusion Feature
        for fusion_idx, conf in enumerate(self.conf):
            image_feat = image_features[fusion_idx]
            audio_feat = audio_features[fusion_idx]

            if fusion_idx == 0:
                fused = torch.cat((image_feat, audio_feat), 1)
                out = self.fusion_layers[fusion_idx](fused)

            else:
                fused = torch.cat((image_feat, audio_feat, out), 1)
                out = self.fusion_layers[fusion_idx](fused)

        # Dropout with Classification
        out = self.central_classifier(out)
        return out

    def central_params(self):
        central_parameters = [
            {'params': self.fusion_layers.parameters()},
            {'params': self.central_classifier.parameters()}
        ]

        return central_parameters

    def _create_fc_layers(self):
        fusion_layers = []

        for i, conf in enumerate(self.conf):
            in_size = sum(self.alphas[i])

            # args.inner_representation_size => ANN Output Size
            if i > 0:
                in_size += self.out_size

            out_size = self.out_size

            # Activation Function
            if conf[2] == 0:
                nl = nn.Sigmoid()
            elif conf[2] == 1:
                nl = nn.ReLU()

            op = nn.Sequential(nn.Linear(in_size, out_size), nl)
            fusion_layers.append(op)

        return nn.ModuleList(fusion_layers)

### Model Train & Evaludation

**Model Train**

In [10]:
def model_train(config, train_loader, val_loader, audio_feature_extractor, imgae_feature_extractor):
    device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
    
    # Define Config in Result txt File
    with open('./result/MFAS.txt', 'a') as f:
        f.write("Config: ")
        for item in config:
            f.write("%s  "%item)
        f.write("\n")
        f.close()
    
    # Define Model
    model = mfas_fusion(np.array(config),audio_feature_extractor, imgae_feature_extractor, device).to(device)
    
    # Hyper Parameter Setting => For Early Stopping
    early_stopping = Utils.EarlyStopping(patience=10, delta=0)
    valid_losses = []
    best_loss = np.Inf
    
    # train
    votenum = 1
    warnings.filterwarnings('ignore')
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    
    total_epoch = 100
    for epoch in tqdm(range(total_epoch), desc='Epoch...'):
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())

        Utils.Late_Fusion_train(optimizer, model, train_loader, device)

        targetlist, scorelist, predlist, epoch_loss = Utils.Late_Fusion_val(model, val_loader, device)
        vote_pred = vote_pred + predlist
        vote_score = vote_score + scorelist

        # major vote
        vote_pred[vote_pred <= (votenum / 2)] = 0
        vote_pred[vote_pred > (votenum / 2)] = 1
        vote_score = vote_score / votenum

        print('vote_pred', vote_pred)
        print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()

        print('TP=', TP, 'TN=', TN, 'FN=', FN, 'FP=', FP)
        print('TP+FP', TP + FP)
        p = TP / (TP + FP)
        print('precision', p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall', r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1', F1)
        print('acc', acc)
        AtrainUC = roc_auc_score(targetlist, vote_score)
        print('AUC: ', AtrainUC)

        # For Early Stopping
        valid_losses.append(epoch_loss.item())
        valid_loss = np.average(valid_losses)
        early_stopping(valid_loss)

        # Best Model Candidate
        if epoch_loss < best_loss:
            best_model_sd = copy.deepcopy(model.state_dict())
            best_loss = epoch_loss
            with open('./result/MFAS.txt', 'a') as file:
                line = 'Validation' + '\n'
                file.write(line)
                line = 'TP=' + str(TP) + 'TN=' + str(TN) + 'FN=' + str(FN) + 'FP=' + str(FP) + '\n'
                file.write(line)
                line = 'Precision: ' + str(p) + ' Recall: ' + str(r) + ' F1: ' + str(F1) + ' ACC: ' + str(
                    acc) + ' AUC: ' + str(AtrainUC) + '\n'
                file.write(line)


        if early_stopping.early_stop:
            print('Early Stopping')
            print('Epoch [{}/{}],  Dev AUC: {:.4f}'.format((epoch + 1), total_epoch, epoch_loss.item()))

            break

    model.load_state_dict(best_model_sd)

    return model

**Result of Config1**

In [11]:
############# Train ##################
train_model = model_train(config_1, multi_train_loader, multi_val_loader, audio_feature_extractor, imgae_feature_extractor)
model_name = str()
for c1 in config_1:
    for c2 in c1:
        model_name+=str(c2)+'_'
model_name+='.pt'

# Save Model
torch.save(train_model.state_dict(), './model/multi_modality/'+model_name)

####################### Test #########################
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())

# Load Pre Train Model
test_model = mfas_fusion(np.array(config_1),audio_feature_extractor, imgae_feature_extractor, device).to(device)
test_model.load_state_dict(torch.load('./model/multi_modality/'+model_name, map_location='cpu'))
test_model.to(device)

# Late Fusion Test
targetlist, scorelist, predlist = Utils.Late_Fusion_test(test_model, multi_test_loader, device)

print('target', targetlist)
print('score', scorelist)
print('predict', predlist)
vote_pred = vote_pred + predlist
vote_score = vote_score + scorelist

TP = ((predlist == 1) & (targetlist == 1)).sum()

TN = ((predlist == 0) & (targetlist == 0)).sum()
FN = ((predlist == 0) & (targetlist == 1)).sum()
FP = ((predlist == 1) & (targetlist == 0)).sum()

print('TP=', TP, 'TN=', TN, 'FN=', FN, 'FP=', FP)
print('TP+FP', TP + FP)
p = TP / (TP + FP)
print('precision', p)
p = TP / (TP + FP)
r = TP / (TP + FN)
print('recall', r)
F1 = 2 * r * p / (r + p)
acc = (TP + TN) / (TP + TN + FP + FN)
print('F1', F1)
print('acc', acc)
AUC = roc_auc_score(targetlist, vote_score)
print('AUC', AUC)

with open('./result/MFAS_Config_1_Result.txt', 'w') as file:
    line = 'TP='+str(TP)+'TN='+str(TN)+'FN='+str(FN)+'FP='+str(FP)+'\n'
    file.write(line)
    line = 'Precision: '+str(p)+' Recall: '+str(r)+' F1: '+str(F1)+' ACC: '+str(acc)+' AUC: '+str(AUC)+'\n'
    file.write(line)

Iteration...: 100it [04:10,  2.50s/it]
Iteration...: 0it [00:00, ?it/s]

vote_pred [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [03:42,  2.22s/it]
Iteration...: 0it [00:00, ?it/s]

vote_pred [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [03:59,  2.40s/it]
Iteration...: 0it [00:00, ?it/s]

...

vote_pred [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [04:40,  2.80s/it]
Iteration...: 0it [00:00, ?it/s]

vote_pred [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [04:02,  2.42s/it]


vote_pred [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

**Result of Config2**

In [12]:
############# Train ##################
train_model = model_train(config_2, multi_train_loader, multi_val_loader, audio_feature_extractor, imgae_feature_extractor)
model_name = str()
for c1 in config_2:
    for c2 in c1:
        model_name+=str(c2)+'_'
model_name+='.pt'

# Save Model
torch.save(train_model.state_dict(), './model/multi_modality/'+model_name)

####################### Test #########################
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())

# Load Pre Train Model
test_model = mfas_fusion(np.array(config_1),audio_feature_extractor, imgae_feature_extractor, device).to(device)
test_model.load_state_dict(torch.load('./model/multi_modality/'+model_name, map_location='cpu'))
test_model.to(device)

# Late Fusion Test
targetlist, scorelist, predlist = Utils.Late_Fusion_test(test_model, multi_test_loader, device)

print('target', targetlist)
print('score', scorelist)
print('predict', predlist)
vote_pred = vote_pred + predlist
vote_score = vote_score + scorelist

TP = ((predlist == 1) & (targetlist == 1)).sum()

TN = ((predlist == 0) & (targetlist == 0)).sum()
FN = ((predlist == 0) & (targetlist == 1)).sum()
FP = ((predlist == 1) & (targetlist == 0)).sum()

print('TP=', TP, 'TN=', TN, 'FN=', FN, 'FP=', FP)
print('TP+FP', TP + FP)
p = TP / (TP + FP)
print('precision', p)
p = TP / (TP + FP)
r = TP / (TP + FN)
print('recall', r)
F1 = 2 * r * p / (r + p)
acc = (TP + TN) / (TP + TN + FP + FN)
print('F1', F1)
print('acc', acc)
AUC = roc_auc_score(targetlist, vote_score)
print('AUC', AUC)

with open('./result/MFAS_Config_2_Result.txt', 'w') as file:
    line = 'TP='+str(TP)+'TN='+str(TN)+'FN='+str(FN)+'FP='+str(FP)+'\n'
    file.write(line)
    line = 'Precision: '+str(p)+' Recall: '+str(r)+' F1: '+str(F1)+' ACC: '+str(acc)+' AUC: '+str(AUC)+'\n'
    file.write(line)

Iteration...: 100it [04:58,  2.98s/it]
Iteration...: 0it [00:00, ?it/s]

vote_pred [0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [05:08,  3.09s/it]
Iteration...: 0it [00:00, ?it/s]

vote_pred [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [06:47,  4.08s/it]
Iteration...: 0it [00:00, ?it/s]

...

Iteration...: 100it [05:04,  3.04s/it]
Iteration...: 0it [00:00, ?it/s]

vote_pred [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [05:09,  3.10s/it]


vote_pred [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

RuntimeError: Error(s) in loading state_dict for mfas_fusion:
	Unexpected key(s) in state_dict: "fusion_layers.2.0.weight", "fusion_layers.2.0.bias". 
	size mismatch for fusion_layers.0.0.weight: copying a param with shape torch.Size([100, 1920]) from checkpoint, the shape in current model is torch.Size([100, 3328]).
	size mismatch for fusion_layers.1.0.weight: copying a param with shape torch.Size([100, 1380]) from checkpoint, the shape in current model is torch.Size([100, 2404]).

In [13]:
####################### Test #########################
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())

# Load Pre Train Model
test_model = mfas_fusion(np.array(config_2),audio_feature_extractor, imgae_feature_extractor, device).to(device)
test_model.load_state_dict(torch.load('./model/multi_modality/'+model_name, map_location='cpu'))
test_model.to(device)

# Late Fusion Test
targetlist, scorelist, predlist = Utils.Late_Fusion_test(test_model, multi_test_loader, device)

print('target', targetlist)
print('score', scorelist)
print('predict', predlist)
vote_pred = vote_pred + predlist
vote_score = vote_score + scorelist

TP = ((predlist == 1) & (targetlist == 1)).sum()

TN = ((predlist == 0) & (targetlist == 0)).sum()
FN = ((predlist == 0) & (targetlist == 1)).sum()
FP = ((predlist == 1) & (targetlist == 0)).sum()

print('TP=', TP, 'TN=', TN, 'FN=', FN, 'FP=', FP)
print('TP+FP', TP + FP)
p = TP / (TP + FP)
print('precision', p)
p = TP / (TP + FP)
r = TP / (TP + FN)
print('recall', r)
F1 = 2 * r * p / (r + p)
acc = (TP + TN) / (TP + TN + FP + FN)
print('F1', F1)
print('acc', acc)
AUC = roc_auc_score(targetlist, vote_score)
print('AUC', AUC)

with open('./result/MFAS_Config_2_Result.txt', 'w') as file:
    line = 'TP='+str(TP)+'TN='+str(TN)+'FN='+str(FN)+'FP='+str(FP)+'\n'
    file.write(line)
    line = 'Precision: '+str(p)+' Recall: '+str(r)+' F1: '+str(F1)+' ACC: '+str(acc)+' AUC: '+str(AUC)+'\n'
    file.write(line)

tensor([[0.0396, 0.9604],
        [0.3990, 0.6010],
        [0.0453, 0.9547],
        [0.9779, 0.0221],
        [0.9641, 0.0359],
        [0.9625, 0.0375],
        [0.2413, 0.7587],
        [0.9816, 0.0184],
        [0.9616, 0.0384],
        [0.9107, 0.0893]], device='cuda:7')
tensor([[0.1697, 0.8303],
        [0.9830, 0.0170],
        [0.9689, 0.0311],
        [0.9667, 0.0333],
        [0.8972, 0.1028],
        [0.9787, 0.0213],
        [0.0787, 0.9213],
        [0.2988, 0.7012],
        [0.9741, 0.0259],
        [0.9820, 0.0180]], device='cuda:7')
tensor([[0.0929, 0.9071],
        [0.9716, 0.0284],
        [0.1146, 0.8854],
        [0.6399, 0.3601],
        [0.9741, 0.0259],
        [0.9632, 0.0368],
        [0.9770, 0.0230],
        [0.8845, 0.1155],
        [0.9830, 0.0170],
        [0.9380, 0.0620]], device='cuda:7')
tensor([[0.9740, 0.0260],
        [0.9766, 0.0234],
        [0.9767, 0.0233],
        [0.5970, 0.4030],
        [0.9767, 0.0233],
        [0.8179, 0.1821],
        [0

**Result of Config3**

In [14]:
############# Train ##################
train_model = model_train(config_3, multi_train_loader, multi_val_loader, audio_feature_extractor, imgae_feature_extractor)
model_name = str()
for c1 in config_3:
    for c2 in c1:
        model_name+=str(c2)+'_'
model_name+='.pt'

# Save Model
torch.save(train_model.state_dict(), './model/multi_modality/'+model_name)

####################### Test #########################
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())

# Load Pre Train Model
test_model = mfas_fusion(np.array(config_3),audio_feature_extractor, imgae_feature_extractor, device).to(device)
test_model.load_state_dict(torch.load('./model/multi_modality/'+model_name, map_location='cpu'))
test_model.to(device)

# Late Fusion Test
targetlist, scorelist, predlist = Utils.Late_Fusion_test(test_model, multi_test_loader, device)

print('target', targetlist)
print('score', scorelist)
print('predict', predlist)
vote_pred = vote_pred + predlist
vote_score = vote_score + scorelist

TP = ((predlist == 1) & (targetlist == 1)).sum()

TN = ((predlist == 0) & (targetlist == 0)).sum()
FN = ((predlist == 0) & (targetlist == 1)).sum()
FP = ((predlist == 1) & (targetlist == 0)).sum()

print('TP=', TP, 'TN=', TN, 'FN=', FN, 'FP=', FP)
print('TP+FP', TP + FP)
p = TP / (TP + FP)
print('precision', p)
p = TP / (TP + FP)
r = TP / (TP + FN)
print('recall', r)
F1 = 2 * r * p / (r + p)
acc = (TP + TN) / (TP + TN + FP + FN)
print('F1', F1)
print('acc', acc)
AUC = roc_auc_score(targetlist, vote_score)
print('AUC', AUC)

with open('./result/MFAS_Config_3_Result.txt', 'w') as file:
    line = 'TP='+str(TP)+'TN='+str(TN)+'FN='+str(FN)+'FP='+str(FP)+'\n'
    file.write(line)
    line = 'Precision: '+str(p)+' Recall: '+str(r)+' F1: '+str(F1)+' ACC: '+str(acc)+' AUC: '+str(AUC)+'\n'
    file.write(line)

Iteration...: 100it [04:36,  2.77s/it]
Iteration...: 0it [00:00, ?it/s]

vote_pred [0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [04:36,  2.77s/it]
Iteration...: 0it [00:00, ?it/s]

vote_pred [0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1.
 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0.
 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [04:53,  2.93s/it]
Iteration...: 0it [00:00, ?it/s]

...

vote_pred [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [04:28,  2.68s/it]
Iteration...: 0it [00:00, ?it/s]

vote_pred [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Iteration...: 100it [05:10,  3.10s/it]


vote_pred [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
targetlist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 